## Python Code for 엘롯데 샬롯_코호트분석 (19년 11월1주차~12월5주차, 주차별 재방문 User)

In [1]:
import datetime
import csv
import glob
import os

# user_logs_* 파일 접근
input_path = os.path.expanduser('~/Desktop/쇼핑어드바이저셀/monthly/input/user_logs/cohort')
input_file = [i for i in glob.glob(os.path.join(input_path, 'user_logs_*'))]
input_file.sort()

# user_logs_* 파일의 user_id 컬럼값 (중복 제거) 저장
user_id_dict = dict()
cnt = 1

for file in input_file:
    with open(file, 'r', newline='') as user_csv_file:
        filereader = csv.reader(user_csv_file)
        #header = next(filereader)
        test_set = set()
        for row in filereader:
            user_id = row[2]
            test_set.add(user_id)
        user_id_dict[cnt] = test_set
        cnt += 1

# cnt : 1 -> 1주차(11월 1주차) ,, 5 -> 5주차(12월 1주차) ,, 9 -> 9주차(12월 5주차)
# 각 주차별 기준으로, 9주차까지의 재방문 숫자 구해서 리스트에 값 넣기
# agn_list -> 재방문 유저 카운트 변수 리스트 (결측치는 0로)
# agn_percent_list -> 재방문 유저 비율(%)
# weekly_cnt -> 각 월 방문 고객의 수
agn_list = [[0 for col in range(9)] for row in range(9)]
agn_percent_list = [[0 for col in range(9)] for row in range(9)]
weekly_cnt = list()

for i in range(1,10):
    ith_weekly_user_list = list(user_id_dict[i])
    weekly_cnt.append(len(ith_weekly_user_list))
    temp_list = ith_weekly_user_list
    temp_list2 = list()
    for j in range(i+1,10):
        ith_next_weekly_user_list = list(user_id_dict[j])
        agn = 0
        for userI in temp_list:
            if userI in ith_next_weekly_user_list:
                agn += 1
                temp_list2.append(userI)
        temp_list = temp_list2
        temp_list2 = []
        agn_list[i-1][j-1] = agn
        
for i in range(1,10):
    for j in range(i+1,10):
        agn_percent_list[i-1][j-1] = round((agn_list[i-1][j-1] / weekly_cnt[i-1])*100, 2)

## Pandas 이용하여 cohort분석 결과 표 만들기

In [2]:
import pandas as pd

#1. 재방문 고객수(명)
df_mdf = pd.DataFrame(agn_list, 
                  index = ["기준_11월 1주차", "기준_11월 2주차", "기준_11월 3주차", 
                           "기준_11월 4주차", "기준_12월 1주차", "기준_12월 2주차",
                           "기준_12월 3주차", "기준_12월 4주차", "기준_12월 5주차"],
                 columns = ["11월 1주차", "11월 2주차", "11월 3주차", 
                            "11월 4주차", "12월 1주차", "12월 2주차", 
                            "12월 3주차", "12월 4주차", "12월 5주차"])

df_mdf['합계'] = df_mdf.iloc[:,0:9].sum(axis=1)
df_mdf['방문고객수'] = weekly_cnt
columnList = ["방문고객수","11월 1주차", "11월 2주차", "11월 3주차",
              "11월 4주차", "12월 1주차", "12월 2주차", "12월 3주차", 
              "12월 4주차", "12월 5주차", "합계"]
df_mdf_f = df_mdf[columnList]
df_mdf_f

,방문고객수,11월 1주차,11월 2주차,11월 3주차,11월 4주차,12월 1주차,12월 2주차,12월 3주차,12월 4주차,12월 5주차,합계
기준_11월 1주차,9663,0,541,84,31,19,12,11,11,8,717
기준_11월 2주차,10730,0,0,587,86,32,20,13,11,8,757
기준_11월 3주차,11325,0,0,0,544,80,26,14,11,8,683
기준_11월 4주차,12705,0,0,0,0,505,75,30,14,10,634
기준_12월 1주차,9134,0,0,0,0,0,428,75,24,16,543
기준_12월 2주차,8814,0,0,0,0,0,0,500,71,30,601
기준_12월 3주차,8754,0,0,0,0,0,0,0,613,181,794
기준_12월 4주차,8977,0,0,0,0,0,0,0,0,842,842
기준_12월 5주차,7461,0,0,0,0,0,0,0,0,0,0


In [3]:
#2. 재방문 비율(%)
df2 = pd.DataFrame(agn_percent_list, 
                  index = ["기준_11월 1주차", "기준_11월 2주차", "기준_11월 3주차", 
                           "기준_11월 4주차", "기준_12월 1주차", "기준_12월 2주차",
                           "기준_12월 3주차", "기준_12월 4주차", "기준_12월 5주차"],
                 columns = ["11월 1주차", "11월 2주차", "11월 3주차", 
                            "11월 4주차", "12월 1주차", "12월 2주차", 
                            "12월 3주차", "12월 4주차", "12월 5주차"])

df2['방문고객수'] = weekly_cnt
columnList = ["방문고객수","11월 1주차", "11월 2주차", "11월 3주차",
              "11월 4주차", "12월 1주차", "12월 2주차", "12월 3주차", 
              "12월 4주차", "12월 5주차"]
df2 = df2[columnList]
df2

,방문고객수,11월 1주차,11월 2주차,11월 3주차,11월 4주차,12월 1주차,12월 2주차,12월 3주차,12월 4주차,12월 5주차
기준_11월 1주차,9663,0,5.6,0.87,0.32,0.20,0.12,0.11,0.11,0.08
기준_11월 2주차,10730,0,0.0,5.47,0.80,0.30,0.19,0.12,0.10,0.07
기준_11월 3주차,11325,0,0.0,0.00,4.80,0.71,0.23,0.12,0.10,0.07
기준_11월 4주차,12705,0,0.0,0.00,0.00,3.97,0.59,0.24,0.11,0.08
기준_12월 1주차,9134,0,0.0,0.00,0.00,0.00,4.69,0.82,0.26,0.18
기준_12월 2주차,8814,0,0.0,0.00,0.00,0.00,0.00,5.67,0.81,0.34
기준_12월 3주차,8754,0,0.0,0.00,0.00,0.00,0.00,0.00,7.00,2.07
기준_12월 4주차,8977,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,9.38
기준_12월 5주차,7461,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
